In [2]:
import pandas as pd
import re
import string
import torch
import transformers
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

C:\Users\CYTech Student\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
%pip install pandas
%pip install transformers
%pip install re
%pip install nltk
%pip install matplotlib
%pip install string
%pip install numpy
%pip install sklearn
%pip install seaborn
%pip install datasets
%pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement string (from versions: none)
ERROR: No matching distribution found for string
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post11-py3-none-any.whl size=2959 sha256=8ad5fb39b4f9266737521f1082941995fa93e8572ea96700e99cedc23d57ec1d
  Stored in directory: /root/.cache/pip/wheels/aa/9c/60/f67813603a52fc35057868f1aba0003cc75b72583dcaa2c341
Successfully built 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h1><strong> Partie I : Fake and real news dataset </strong></h1>

<h2><strong> Data Exploration </strong></h2>

In [ ]:
df1 = pd.read_csv("drive/MyDrive/True.csv")
df1["labels"] = 1

In [7]:
df1.columns

Index(['title', 'text', 'subject', 'date', 'labels'], dtype='object')

In [6]:
df2 = pd.read_csv("drive/MyDrive/Fake.csv")
df2["labels"] = 0

In [8]:
df2.columns

Index(['title', 'text', 'subject', 'date', 'labels'], dtype='object')

In [9]:
df = pd.concat([df1, df2], axis=0)

In [ ]:
df.columns

Index(['title', 'text', 'subject', 'date', 'labels'], dtype='object')

In [10]:
df["text"] = df["title"] + df["text"]
df = df.drop(['date', 'title', 'subject'], axis=1)

In [ ]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = transformers.DistilBertModel.from_pretrained("distilbert-base-uncased")

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [ ]:
datasets = dataset.train_test_split(test_size=0.2)

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 35918
    })
    test: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 8980
    })
})

In [ ]:
labels = [label for label in datasets['train'].features.keys() if label not in ['text', '__index_level_0__']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['labels']

In [ ]:
labels = [label for label in datasets['train'].features.keys() if label not in ['text', '__index_level_0__']]
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", padding=False, truncation=True)

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

In [ ]:
encoded_dataset = datasets.map(preprocess_data, batched=True, remove_columns=datasets['train'].column_names)

Map:   0%|          | 0/35918 [00:00<?, ? examples/s]

Map:   0%|          | 0/8980 [00:00<?, ? examples/s]

In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['labels', 'input_ids', 'attention_mask'])


In [ ]:
tokenizer.decode(example['input_ids'])

'[CLS] what dan rather has to say about trump ’ s ‘ deadly serious ’ flynn scandal is ‘ chilling ’ on tuesday afternoon, legendary journalist dan rather weighed in on the trump administration s latest scandal involving former national security advisor michael flynn s collusion with russia. watergate is the biggest political scandal of my lifetime, until maybe now. it was the closest we came to a debilitating constitutional crisis, until maybe now, rather began. he explained that while he rates watergate as a 9, and the flynn scandal is only at a 5 or 6 right now, the situation is cascading in intensity seemingly by the hour. in [SEP]'

<h2><strong> DistilBert-base-uncased </strong></h2>

In [ ]:
encoded_dataset.set_format("torch")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"distilbert-bse-uncased",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    #roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    #accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average
               #,'roc_auc': roc_auc,
               #'accuracy': accuracy
               }
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [ ]:
encoded_dataset['train'][0]['labels']

tensor([0.])

In [ ]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  2054,  4907,  2738,  2038,  2000,  2360,  2055,  8398,  1521,
         1055,  1520,  9252,  3809,  1521, 13259,  9446,  2003,  1520, 27017,
         1521,  2006,  9857,  5027,  1010,  8987,  4988,  4907,  2738, 12781,
         1999,  2006,  1996,  8398,  3447,  1055,  6745,  9446,  5994,  2280,
         2120,  3036,  8619,  2745, 13259,  1055,  8902, 24117,  2007,  3607,
         1012,  2300,  5867,  2003,  1996,  5221,  2576,  9446,  1997,  2026,
         6480,  1010,  2127,  2672,  2085,  1012,  2009,  2001,  1996,  7541,
         2057,  2234,  2000,  1037,  2139, 14454, 16518,  6543,  5325,  1010,
         2127,  2672,  2085,  1010,  2738,  2211,  1012,  2002,  4541,  2008,
         2096,  2002,  6165,  2300,  5867,  2004,  1037,  1023,  1010,  1998,
         1996, 13259,  9446,  2003,  2069,  2012,  1037,  1019,  2030,  1020,
         2157,  2085,  1010,  1996,  3663,  2003, 25222,  3540,  4667,  1999,
         8015,  9428,  2011,  1996,  3178,  1012,  1999,   102])

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7338, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[0.0797]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
encoded_dataset["test"]["labels"]

tensor([[1.],
        [0.],
        [1.],
        ...,
        [0.],
        [1.],
        [0.]])

In [ ]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,0.006500,0.000315,0.999889
2,0.003200,0.000621,0.999889
3,0.000000,0.000164,0.999889
4,0.000000,0.000004,1.000000
5,0.000000,0.000001,1.000000


TrainOutput(global_step=22450, training_loss=0.0033489484240908697, metrics={'train_runtime': 951.4931, 'train_samples_per_second': 188.745, 'train_steps_per_second': 23.594, 'total_flos': 5947348966740480.0, 'train_loss': 0.0033489484240908697, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 4.166920916759409e-06,
 'eval_f1': 1.0,
 'eval_runtime': 12.8142,
 'eval_samples_per_second': 700.785,
 'eval_steps_per_second': 87.637,
 'epoch': 5.0}

<h2><strong> Bert </strong></h2>

In [11]:
df.head()

,text,labels
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [12]:
%pip install transformers

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch

df_sampled = df.sample(frac=0.1, random_state=42)

# Séparer les données en ensemble d'entraînement et de test
train_df, test_df = train_test_split(df_sampled, test_size=0.2, random_state=42)

# Charger le tokenizer et le modèle
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # 2 labels: fake ou vrai

# Tokeniser les données
def tokenize_data(df, tokenizer, max_length=512):
    tokens = tokenizer.batch_encode_plus(
        df['text'].tolist(),
        max_length=max_length,
        pad_to_max_length=True,
        truncation=True,
        return_tensors='pt'
    )
    return tokens

train_tokens = tokenize_data(train_df, tokenizer)
test_tokens = tokenize_data(test_df, tokenizer)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [16]:
# Créer des datasets PyTorch
train_dataset = TensorDataset(train_tokens['input_ids'], train_tokens['attention_mask'], torch.tensor(train_df['labels'].tolist()))
test_dataset = TensorDataset(test_tokens['input_ids'], test_tokens['attention_mask'], torch.tensor(test_df['labels'].tolist()))

# DataLoader pour itérer sur les données
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Paramètres d'entraînement
optimizer = AdamW(model.parameters(), lr=5e-5)
epochs = 3

# Entraînement du modèle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()

# Évaluation du modèle
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calcul de l'accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy}")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Accuracy: 0.9977728285077951


<h2><strong>Logistic Regression</strong></h2>

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


# Séparer les données en ensemble d'entraînement et de test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Créer un vecteurisateur Bag of Words
vectorizer = CountVectorizer(max_features=5000)  # Nombre maximum de mots à prendre en compte

# Transformer les textes en vecteurs
X_train = vectorizer.fit_transform(train_df['text'])
X_test = vectorizer.transform(test_df['text'])

# Définir les étiquettes
y_train = train_df['labels']
y_test = test_df['labels']

# Créer un modèle de régression logistique
model = LogisticRegression()

# Entraîner le modèle
model.fit(X_train, y_train)

# Faire des prédictions
predictions = model.predict(X_test)

# Calculer l'accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

f1 = f1_score(y_test, predictions)
print(f"f1": {f1}")


Accuracy: 0.9963251670378619


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


<h2><strong> Conclusion </strong></h2>

Les résultats des trois modèles de prédiction sont trop élèvé sans optimisation. J'en conclu que le dataset Fake ans real news est trop simple. Nous avons donc fait le choix de prendre un autre jeu de données : FakeNewsChallenge